# Automatic algorithm/strategy selection
Here we will show you how you can perform an algorithm selection, helping you choose the best optimization algorithm for your specific problem.

In [1]:
from metaheuristic_designer import simple
from metaheuristic_designer.operators import OperatorVector
from metaheuristic_designer.selectionMethods import ParentSelection, SurvivorSelection
from metaheuristic_designer.algorithms import AlgorithmSelection, StrategySelection
from metaheuristic_designer.initializers import UniformVectorInitializer
from metaheuristic_designer.strategies import HillClimb, SA, ES, GA, DE, PSO, RandomSearch
from metaheuristic_designer.benchmarks import *
import numpy as np

For the following examples, we will use the Rastrigin function for which we need to find the minimum value. This is a very hard optimization problem and will allow us to compare the performance of different algorithms.

In [2]:
rastrigin_func = Rastrigin(size=2)

## Algorithm selection
We will first explore the AlgorithmSelection class, which launches each of the specified algorithms multiple times and gets some statistics about the evaluation.

We begin by defining the algorithm we are going to use. The ```simple``` package gives us premade implementations of well known optimization algorithms with a single function call, although customization is limited.

In [3]:
# The parameters common to all algorithms
params = {
    # Stopping condition
    "stop_cond": "neval",
    "neval": 5e4,
    # Encoding
    "encoding": "real",
    # Disable verbose, the output gets very cluttered otherwise
    "verbose": False,
    # Number of times to launch each algorithm
    "repetitions": 10,
}

# Define the algorithms we are going to use
algorithms = [
    simple.random_search(params, rastrigin_func),
    simple.hill_climb(params, rastrigin_func),
    simple.simulated_annealing(params, rastrigin_func),
    simple.evolution_strategy(params, rastrigin_func),
    simple.genetic_algorithm(params, rastrigin_func),
    simple.differential_evolution(params, rastrigin_func),
    simple.particle_swarm(params, rastrigin_func),
]

# Instanciate the AlgorithmSelection class
algorithm_search = AlgorithmSelection(algorithms, params)

# Launch all the algorithms and get the best overall solution
solution, best_fitness, report = algorithm_search.optimize()

KeyboardInterrupt: 

The optimize function should have returned the best overall solution and a report with statistics about the execution

In [ ]:
print(f"best solution: {solution}")
print(f"fitness: {best_fitness}")

In [ ]:
report.sort_values("fitness_avg")

With this, we can clearly see that the best algorithm for this task seems to be the Differential Evolution algorithm, and surprisingly, RandomSearch performs better than HillClimb, SA, ES and even GA. This could be caused by some misconfiguration of parameters, but since we just used the default implementation, we have no way of changing this.

## Strategy Selection
We could have specified the algorithms directly constructing them piece by piece, but unless you need to try some specific implementation of an algorithm, it is much simpler to search for the best Search Strategy. This is where the StrategySearch class is used. It is very similar to the AlgorithmSelection class, but using search strategies.

This also has the advantage that you can choose a custom name for each strategy.

In [ ]:
# Initializer for sinlge-solution algorithms
single_initializer = UniformVectorInitializer(rastrigin_func.vecsize, rastrigin_func.low_lim, rastrigin_func.up_lim, pop_size=1)

# Initializer for population-based algorithms
pop_initializer = UniformVectorInitializer(rastrigin_func.vecsize, rastrigin_func.low_lim, rastrigin_func.up_lim, pop_size=100)

# Define strategies to be tested
strategies = [
    RandomSearch(pop_initializer),
    HillClimb(
        single_initializer,
        OperatorVector("RandNoise", {"distrib": "Gauss", "F": 1e-3}),
        name="HillClimb",
    ),
    SA(
        single_initializer,
        OperatorVector("RandNoise", {"distrib": "Gauss", "F": 1e-3}),
        params={"iter": 100, "temp_init": 1, "alpha": 0.995},
        name="SA",
    ),
    SA(
        pop_initializer,
        OperatorVector("RandNoise", {"distrib": "Gauss", "F": 1e-3}),
        params={"iter": 100, "temp_init": 1, "alpha": 0.995},
        name="ParallelSA",
    ),
    ES(
        pop_initializer,
        mutation_op=OperatorVector("RandNoise", {"distrib": "Gauss", "F": 5e-3}),
        survivor_sel=SurvivorSelection("(m+n)"),
        params={"offspringSize": 150},
        name="ES-(μ+λ)",
    ),
    ES(
        pop_initializer,
        mutation_op=OperatorVector("RandNoise", {"distrib": "Gauss", "F": 5e-2}),
        survivor_sel=SurvivorSelection("(m,n)"),
        params={"offspringSize": 700},
        name="ES-(μ,λ)",
    ),
    GA(
        pop_initializer,
        mutation_op=OperatorVector("RandNoise", {"distrib": "Gauss", "F": 1e-3}),
        cross_op=OperatorVector("Multipoint"),
        parent_sel=ParentSelection("Tournament", {"amount": 60, "p": 0.1}),
        survivor_sel=SurvivorSelection("Elitism", {"amount": 10}),
        params={"pcross": 0.9, "pmut": 0.1},
        name="GA",
    ),
    DE(
        pop_initializer,
        OperatorVector("DE/best/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/best/1",
    ),
    DE(
        pop_initializer,
        OperatorVector("DE/rand/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/rand/1",
    ),
    DE(
        pop_initializer,
        OperatorVector("DE/current-to-best/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/current-to-best/1",
    ),
    PSO(pop_initializer, {"w": 0.7, "c1": 1.5, "c2": 1.5}, name="PSO"),
]

# Instanciate the StrategySelection class
algorithm_search = StrategySelection(
    rastrigin_func,
    strategies,
    algorithm_params={
        "stop_cond": "neval",
        "neval": 5e4,
        "verbose": False,
    },
    params={"verbose": True, "repetitions": 10},
)

# Launch all the algorithms and get the best overall solution
solution, best_fitness, report = algorithm_search.optimize()

In [ ]:
print(f"best solution: {solution}")
print(f"fitness: {best_fitness}")

In [ ]:
report.sort_values("fitness_avg")